In [1]:
from __future__ import division
import numpy as np
from sklearn import datasets, linear_model
import pandas as pd
from pandas import DataFrame, Series
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
sns.set(style='ticks', palette='Set2')
%matplotlib inline

NameError: name 'sns' is not defined

In [ ]:
# correlated columns
# Suppose p = 5, n = 500 -- a good ratio
p = 5; n = 500
X = np.random.randn(n,p)
c = np.dot(X[:,0:p-1],[2,3,-1,0.5]) + np.random.randn(1,n)/1000;
# c is a column that is approximately a linear combination of
# columns 1, 2, 3 and 4. We set the 5th column of X equal to c
X[:,p-1] = c
beta = np.array([1,1,1,1,0])
sigma = 0.3333
y = np.dot(X,beta) + sigma*np.random.randn(n)
# Now let's also generate a test data set
n1 = 500 # number of testing points
X_test = np.random.randn(n1,p)
# In the test set, there's more noise in the 5th column
c_test = np.dot(X_test[:,0:p-1],[2,3,-1,0.5]) + np.random.randn(1,n1)/50;
X_test[:,p-1] = c_test
y_test = np.dot(X_test,beta) + sigma*np.random.randn(n1)

2. What is the test error of the “zero-variance” solution, namely, the all-zeros solution? (Note that due to the random numbers generated, your solution will not be identical every time you run it, or if you compare with another homework group. You may consider fixing the random seed).

See below. For our run, 45.229 is the approximate test error.

In [6]:
#Beta_hat_R using all zeros solution
beta_hat_R = [0, 0, 0, 0, 0]

# Finally, let's compute the test error for the ridge regression solution.
np.linalg.norm(np.dot(X_test,beta_hat_R) - y_test)

45.228743961847705

3. The least-squares solution does not seem to do too well, because it has so much variance. Still, it is unbiased. Show this empirically: generate many copies of the data, and for each one, obtain the least-squares solution. Average these, to show that while each run produces a beta hat that is very different, their average begins to look more and more like the true beta.

See below for 10000x run, showing recovered beta of [0.9983274799279912, 0.9977620841953185, 1.000794816886055, 0.9996759282628082, 0.0007649002087681994] in our test run.

In [19]:
beta_hat_avg = [0, 0, 0, 0, 0]

for i in range(0, 10000):
    # Now let's also generate a test data set
    n1 = 500 # number of testing points
    X_test = np.random.randn(n1,p)
    # In the test set, there's more noise in the 5th column
    c_test = np.dot(X_test[:,0:p-1],[2,3,-1,0.5]) + np.random.randn(1,n1)/50;
    X_test[:,p-1] = c_test
    y_test = np.dot(X_test,beta) + sigma*np.random.randn(n1)
    # Note that the matrix Z = X'X has a very small eigenvalue.
    Z = np.dot(X_test.T,X_test)
    np.linalg.eig(Z)[0]
    beta_hat = np.dot(np.dot(np.linalg.inv(Z),X_test.T),y_test.T)
    
    for j in range(0, 5):
        beta_hat_avg[j] = ((beta_hat_avg[j] * i) + beta_hat[j]) / (i + 1)

print(beta_hat_avg)

[0.9983274799279912, 0.9977620841953185, 1.000794816886055, 0.9996759282628082, 0.0007649002087681994]


4. Alternatively, if one had access to lots of data, instead of computing the least-square solution over smaller batches and then averaging these solutions as in the previous part of the problem, an approach is to run a single least-squares regression over all the data. Which approach do you think is better? Can you support your conclusion with experiments?

We have already considered an experiment above in which we run an OLS solution over 500 points 10000x. An easy way to compare would be to generate a set of 500 * 10000 = 5000000 points and compare the OLS solution we get from that data. That experiment is shown below.

The difference from true beta is slightly lower with the smaller dataset averaged many times. The difference appears to be nearly an order of magnitude (error in the 0.001 range vs error in the 0.01 range) better. So the conclusion would be that it would be better to average smaller samples than it would be to run OLS on one large, complete dataset.

In [21]:
# Generate large data set equivalent to above experiment for #3 and compute beta_hat

n1 = 5000000 # number of testing points
X_test = np.random.randn(n1,p)
# In the test set, there's more noise in the 5th column
c_test = np.dot(X_test[:,0:p-1],[2,3,-1,0.5]) + np.random.randn(1,n1)/50;
X_test[:,p-1] = c_test
y_test = np.dot(X_test,beta) + sigma*np.random.randn(n1)
# Note that the matrix Z = X'X has a very small eigenvalue.
Z = np.dot(X_test.T,X_test)
np.linalg.eig(Z)[0]
beta_hat = np.dot(np.dot(np.linalg.inv(Z),X_test.T),y_test.T)

print(beta_hat)

[0.98558886 0.97814657 1.00716553 0.99640658 0.00723274]
